# 基于 Excel 的 QA 资料

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os

In [2]:
from textlong import idea
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings
from textlong.knowledge import LocalFilesLoader, QAExcelsLoader, collect_docs

## 加载文档

In [3]:
qa = QAExcelsLoader("QA")
qa.load()

[Document(page_content='低层建筑 \n场景：大于300平方米的银行营业厅;问题：是否需要设置自动喷水系统？', metadata={'answer': '问题答案：需要设立自动喷水灭火系统。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。&#10;&#10;', 'source': 'QA/demo.xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='低层建筑 \n场景：大于300平方米的银行营业厅&#10;问题：&#10;（1）是不是人员密集场所？&#10;（2）依据哪条规范？', metadata={'answer': '问题（1）答案：是人员密集场所。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条明确了人员密集场所的定义，并要求这类场所应采取相应的防火措施。银行营业厅通常会有较多的客户和工作人员，因此属于人员密集场所。&#10;&#10;问题（2）答案：《建筑设计防火规范》（GB50016-2014）。&#10;&#10;相关规范解释：该规范是中国建筑设计和消防审核的重要标准之一，其中包含了关于建筑分类、防火分区、安全疏散、消防设施等方面的详细规定，是判断一个场所是否为人员密集场所的重要依据之一。', 'source': 'QA/demo.xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='低层建筑 \n场景：5层高的大学实验楼&#10;（主要是化工、物理等实验室，并且采用了分体式空调）&#10;问题：是否需要设置自动喷淋系统吗，还是局部设置气体灭火系统？？', metadata={'answer': '问题答案：需要设置自动喷淋系统。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001）规定，大学实验楼作为教育科研建筑，尤其是含有化工、物理等实验室，存在较高的火灾风险，应设置自动喷淋系统。此外，根据实际情况评估，可考虑在特

## 构建缓存

In [4]:
from langchain_zhipu import ZhipuAIEmbeddings
model = ZhipuAIEmbeddings()
qa.cache_embeddings(model)

No embeddings to cached!


False

## 加载文本嵌入缓存

In [5]:
emb, _m, metadata = qa.load_embeddings()

## 基于文本嵌入缓存查询

In [6]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_embeddings(emb, model, metadata)
retriever = db.as_retriever()

In [7]:
print((retriever|collect_docs).invoke('实验楼要不要设置喷淋'))

低层建筑 
场景：5层高的大学实验楼&#10;（主要是化工、物理等实验室，并且采用了分体式空调）&#10;问题：是否需要设置自动喷淋系统吗，还是局部设置气体灭火系统？？
问题答案：需要设置自动喷淋系统。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001）规定，大学实验楼作为教育科研建筑，尤其是含有化工、物理等实验室，存在较高的火灾风险，应设置自动喷淋系统。此外，根据实际情况评估，可考虑在特定区域增设气体灭火系统作为补充。&#10;
-----------------------------------
低层建筑 
场景：大于300平方米的银行营业厅;问题：是否需要设置自动喷水系统？
问题答案：需要设立自动喷水灭火系统。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。&#10;&#10;
-----------------------------------
低层建筑 
场景：大于300平方米的银行营业厅&#10;问题：&#10;（1）是不是人员密集场所？&#10;（2）依据哪条规范？
问题（1）答案：是人员密集场所。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条明确了人员密集场所的定义，并要求这类场所应采取相应的防火措施。银行营业厅通常会有较多的客户和工作人员，因此属于人员密集场所。&#10;&#10;问题（2）答案：《建筑设计防火规范》（GB50016-2014）。&#10;&#10;相关规范解释：该规范是中国建筑设计和消防审核的重要标准之一，其中包含了关于建筑分类、防火分区、安全疏散、消防设施等方面的详细规定，是判断一个场所是否为人员密集场所的重要依据之一。


# RAG 查询

In [3]:
from textlong.memory import MemoryManager, WithMemoryBinding
from textlong.knowledge import LocalFilesLoader, collect_docs
from textlong.chain import create_qa_chain
from langchain_community.vectorstores import FAISS
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings

qa = QAExcelsLoader("QA")
es = qa.load_embeddings(ZhipuAIEmbeddings())
db = FAISS.from_embeddings(*es)

**不使用记忆：**

In [4]:
chain = create_qa_chain(ChatZhipuAI(), db.as_retriever())
for x in chain.stream("实验楼要不要设置喷淋？"):
    print(x.content, end="")

问题答案：需要设置自动喷淋系统。

相关规范解释：《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001）规定，含有化工、物理等实验室的建筑应设置自动喷淋系统，针对较高火灾风险的教育科研建筑有此要求。

**使用默认对话ID(default)：**

In [5]:
memory = MemoryManager()
chain_with_memory = create_qa_chain(ChatZhipuAI(), db.as_retriever(), memory)
for x in chain_with_memory.stream({"input": "银行大厅要不要设置喷淋？"}):
    print(x.content, end="")

问题答案：需要设立自动喷水灭火系统。

相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。

**指定对话ID(1)：**

In [6]:
config = {"configurable": {"session_id": "1"}}
for x in chain_with_memory.stream({"input": "宿舍楼要不要设置喷淋？"}, config):
    print(x.content, end="")

问题答案：需要。

相关规范解释：《建筑设计防火规范》（GB 50016-2014）对宿舍楼等居住建筑也有相应的规定，要求根据建筑的高度、使用人数等因素，设置必要的消防设施，其中包括自动喷淋系统。因此，宿舍楼应根据实际情况判断是否需要设置喷淋系统。

**检查历史对话：**

In [7]:
memory.get_memory("default")

[HumanMessage(content='银行大厅要不要设置喷淋？'),
 AIMessageChunk(content='问题答案：需要设立自动喷水灭火系统。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1719233030, 'index': 0, 'usage': {'prompt_tokens': 567, 'completion_tokens': 68, 'total_tokens': 635}}, id='8780910725273935738')]

In [8]:
memory.get_memory("1")

[HumanMessage(content='宿舍楼要不要设置喷淋？'),
 AIMessageChunk(content='问题答案：需要。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）对宿舍楼等居住建筑也有相应的规定，要求根据建筑的高度、使用人数等因素，设置必要的消防设施，其中包括自动喷淋系统。因此，宿舍楼应根据实际情况判断是否需要设置喷淋系统。', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1719233076, 'index': 0, 'usage': {'prompt_tokens': 567, 'completion_tokens': 67, 'total_tokens': 634}}, id='8780910759633682404')]